In [14]:
from litellm import completion
import os
import docx
import json
from dotenv import load_dotenv
import jinja2
import pandas as pd
import re
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [12]:
def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
    return text

v1 = extract_text_from_docx('data/v1.docx')
v2 = extract_text_from_docx('data/v2.docx')

# count number of tokens in each version
v1_tokens = len(v1.split())/0.75
v2_tokens = len(v2.split())/0.75
print(f"v1 tokens: {v1_tokens}, v2 tokens: {v2_tokens}")

v1 tokens: 20532.0, v2 tokens: 11181.333333333334


In [ ]:
# load prompt from jinja2 template
with open('templates/system_prompt.jinja2') as f:
    system_prompt = jinja2.Template(f.read()).render(v1=v1, v2=v2)

response = completion(
    model="gpt-4o-mini",  # Gemini model = gemini/gemini-1.5-pro
    messages=[{"role": "user", "content": system_prompt}],

)
response = response.choices[0].message.content

In [18]:
print(response)

Here is the detailed analysis and recommendations based on the comparison between the two versions of Apple's Terms & Conditions for Apple Music.

### 1. Key Differences Analysis

| **Change Type**          | **Section**                                   | **Old Text**                                                                                                                       | **New Text**                                                                                                                     |
|--------------------------|-----------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------|
| **Substantive Change**   | Introduction to Services                      | "By creating an account for use of the Services in a particular c

In [21]:
# Extract the table portion from the response
table_text = response.split("## 1. Key Differences Analysis")[1].split("## 2. Impact Assessment")[0]

# Extract rows from the Markdown table
rows = table_text.strip().split("\n")

# Filter only the rows that contain actual table data (skip headers and separators)
table_data = [row.split("|")[1:-1] for row in rows if "|" in row and "---" not in row]

# Remove leading and trailing whitespace from each cell
clean_table = [[cell.strip() for cell in row] for row in table_data]

# Ensure all rows have exactly 5 columns
clean_table = [row if len(row) == 5 else row + [""] * (5 - len(row)) for row in clean_table]

# Create a DataFrame
df = pd.DataFrame(clean_table[1:], columns=clean_table[0])  # First row as headers

# Propagate category values downward to fill missing entries
df["Category"] = df["Category"].replace("", None).ffill()
df




| **Change Type**          | **Section**                                   | **Old Text**                                                                                                                       | **New Text**                                                                                                                     |
|--------------------------|-----------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------|
| **Substantive Change**   | Introduction to Services                      | "By creating an account for use of the Services in a particular country or territory you are specifying it as your Home Country." | "By creating an account for use of the Services in a particular country or territory you specify it as your Hom

KeyError: 'Change Type'